<a href="https://colab.research.google.com/github/jay-kanakia/GenAI/blob/main/10_Pytorch_project_cnn_fashion_mnist_pytorch_gpu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset,DataLoader

In [2]:
!mkdir -p ~/.kaggle
!mv kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

!kaggle datasets download -d zalando-research/fashionmnist
!unzip fashionmnist.zip -d data/

Dataset URL: https://www.kaggle.com/datasets/zalando-research/fashionmnist
License(s): other
  0% 0.00/68.8M [00:00<?, ?B/s]
100% 68.8M/68.8M [00:00<00:00, 1.69GB/s]
Archive:  fashionmnist.zip
  inflating: data/fashion-mnist_test.csv  
  inflating: data/fashion-mnist_train.csv  
  inflating: data/t10k-images-idx3-ubyte  
  inflating: data/t10k-labels-idx1-ubyte  
  inflating: data/train-images-idx3-ubyte  
  inflating: data/train-labels-idx1-ubyte  


In [3]:
torch.manual_seed(42)

In [4]:
train_data=pd.read_csv(r'/content/data/fashion-mnist_train.csv')
test_data=pd.read_csv(r'/content/data/fashion-mnist_test.csv')

train_data.sample(2)

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
36691,8,0,0,0,0,0,0,0,0,0,...,59,55,63,73,103,103,93,81,0,0
4067,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
device=torch.device('cuda')

In [6]:
X_train=train_data.iloc[:,1:].values
X_test=test_data.iloc[:,1:].values

y_train=train_data.iloc[:,0].values
y_test=test_data.iloc[:,0].values

In [7]:
X_train=X_train/255
X_test=X_test/255

X_train_tensor=torch.from_numpy(X_train.astype(np.float32))
X_test_tensor=torch.from_numpy(X_test.astype(np.float32))

y_train_tensor=torch.from_numpy(y_train).long()
y_test_tensor=torch.from_numpy(y_test).long()

In [8]:
class MyDataset(Dataset):

  def __init__(self,features,labels):
    self.features=features.reshape(-1,1,28,28)
    self.labels=labels

  def __len__(self):
    return self.features.shape[0]

  def __getitem__(self,index):
    return self.features[index],self.labels[index]

In [9]:
train_dataset=MyDataset(X_train_tensor,y_train_tensor)
test_dataset=MyDataset(X_test_tensor,y_test_tensor)

In [10]:
train_loader=DataLoader(train_dataset,batch_size=256,shuffle=True)
test_loader=DataLoader(test_dataset,batch_size=256,shuffle=False)

In [11]:
from torch.nn.modules.pooling import MaxPool2d
from torch.nn.modules.batchnorm import BatchNorm2d
class MyNN(nn.Module):

  def __init__(self,input_features):
    super().__init__()

    self.features=nn.Sequential(
        nn.Conv2d(input_features,32,kernel_size=(3,3),padding='same'),
        nn.ReLU(),
        nn.BatchNorm2d(32),
        nn.MaxPool2d(kernel_size=(2,2),stride=(2,2)),

        nn.Conv2d(32,64,kernel_size=(3,3),padding='same'),
        nn.ReLU(),
        nn.BatchNorm2d(64),
        nn.MaxPool2d(kernel_size=(2,2),stride=(2,2))
    )

    self.classifier=nn.Sequential(
        nn.Flatten(),
        nn.Linear(64*7*7,128),
        nn.BatchNorm1d(128),
        nn.Dropout(p=0.3),

        nn.Linear(128,64),
        nn.BatchNorm1d(64),
        nn.Dropout(p=0.3),

        nn.Linear(64,10)

    )

  def forward(self,X):
    x=self.features(X)
    x=self.classifier(x)

    return x

In [12]:
device=torch.device('cuda')

In [13]:
model=MyNN(1)
model=model.to(device)

learning_rate=0.01
epochs=125

optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)
loss_function=nn.CrossEntropyLoss()

In [14]:
for epoch in range(epochs):
  total_loss=0
  for batch_features,batch_labels in train_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    y_pred=model(batch_features)
    loss=loss_function(y_pred,batch_labels)
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    total_loss+=loss.item()

  print(f"Epoch : {epoch+1}, Avg.Loss : {total_loss/len(train_loader)}")

Epoch : 1, Avg.Loss : 0.4074471192157015
Epoch : 2, Avg.Loss : 0.28607058448994416
Epoch : 3, Avg.Loss : 0.2498743141585208
Epoch : 4, Avg.Loss : 0.22672485785281404
Epoch : 5, Avg.Loss : 0.20824210716688887
Epoch : 6, Avg.Loss : 0.19969808376849965
Epoch : 7, Avg.Loss : 0.18584158934177236
Epoch : 8, Avg.Loss : 0.1802398979346803
Epoch : 9, Avg.Loss : 0.16747716830131856
Epoch : 10, Avg.Loss : 0.15780124359942496
Epoch : 11, Avg.Loss : 0.14966271773297735
Epoch : 12, Avg.Loss : 0.14075232290841164
Epoch : 13, Avg.Loss : 0.14096845753015355
Epoch : 14, Avg.Loss : 0.12993226441297126
Epoch : 15, Avg.Loss : 0.124645070208514
Epoch : 16, Avg.Loss : 0.11949818375262808
Epoch : 17, Avg.Loss : 0.11041710890353994
Epoch : 18, Avg.Loss : 0.11012872257765303
Epoch : 19, Avg.Loss : 0.10597978714615741
Epoch : 20, Avg.Loss : 0.100266926228366
Epoch : 21, Avg.Loss : 0.09578929633853284
Epoch : 22, Avg.Loss : 0.0924191970932991
Epoch : 23, Avg.Loss : 0.08959912110833411
Epoch : 24, Avg.Loss : 0.081

In [15]:
model.eval()

MyNN(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
    (5): ReLU()
    (6): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (7): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=3136, out_features=128, bias=True)
    (2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Dropout(p=0.3, inplace=False)
    (4): Linear(in_features=128, out_features=64, bias=True)
    (5): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): 

In [17]:
with torch.no_grad():
  total=0
  correct=0
  for batch_features,batch_labels in test_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    y_pred=model(batch_features)
    y_pred=torch.argmax(y_pred,dim=1)
    total+=batch_labels.shape[0]
    correct+=(y_pred==batch_labels).sum().item()

  print(f"accuracy is {correct/total}")

accuracy is 0.9079


In [19]:
with torch.no_grad():
  total=0
  correct=0
  for batch_features,batch_labels in train_loader:
    batch_features=batch_features.to(device)
    batch_labels=batch_labels.to(device)

    y_pred=model(batch_features)
    y_pred=torch.argmax(y_pred,dim=1)
    total+=batch_labels.shape[0]
    correct+=(y_pred==batch_labels).sum().item()

  print(f"accuracy is {correct/total}")

accuracy is 0.9865666666666667
